In [2]:
%matplotlib widget
import os
import sys
import numpy as np
import matplotlib
import serial
import pyvisa
import csv
import struct as struct
from time import sleep, time
from pylab import *
from datetime import datetime

from scipy import integrate
from scipy.special import jv

import pylab as pyplt
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.interpolate import interp2d

import codecs
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm

In [4]:
def readMiniVNAMeasurements(fileDir,fileName,portdeg):
    results = []
    freq = [] #in Hz
    returnLoss = [] #in dB
    impedanceReal = [] #in Ohms
    impedanceImag = [] # in Ohms
    phase = [] #in Degrees
    with open(fileDir + fileName, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        #next(csvreader)
        #next(csvreader)
        for row in csvreader:
             results.append(row)
    for i in range(len(results)):
        freq.append(float(results[i][0]))
        returnLoss.append(float(results[i][1]))
        phase.append(float(results[i][2]))
        impedanceReal.append(float(results[i][3]))
        impedanceImag.append(float(results[i][5]))
        
    
    phase = adjustPhase(phase,portdeg,freq)
    
    return freq, returnLoss, phase, impedanceReal, impedanceImag

def readMiniVNAMeasurementsSParam(fileDir,fileName,portdeg):
    results = []
    freq = [] #in Hz
    sParam = [] #in dB
    impedanceReal = [] #in Ohms
    impedanceImag = [] # in Ohms
    phase = [] #in Degrees
    with open(fileDir + fileName, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        #next(csvreader)
        #next(csvreader)
        for row in csvreader:
             results.append(row)
    for i in range(len(results)):
        freq.append(float(results[i][0]))
        sParam.append(10**(float(results[i][1])/20))
        phase.append(float(results[i][2]))
        impedanceReal.append(float(results[i][3]))
        impedanceImag.append(float(results[i][5]))
        
    
    phase = adjustPhase(phase,portdeg,freq)
    
    return freq, sParam, phase, impedanceReal, impedanceImag


def readSimMeasurements(fileDir,fileName):
    results = []
    freq = [] #in Hz
    returnLoss = [] #in dB
    phase = [] #in Degrees
    with open(fileDir + fileName, newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        #next(csvreader)
        #next(csvreader)
        for row in csvreader:
             results.append(row)
    for i in range(len(results)):
        freq.append(float(results[i][1]))
        returnLoss.append(float(results[i][2]))
    
    return freq, returnLoss



def adjustPhase(array,portdeg,freq):   
    for i in range(len(array)):
        out0 = 18.025*1e-3/3e8*freq[i]*360
        out90 = 10.727*1e-3/3e8*freq[i]*360
        out180 = 12.139*1e-3/3e8*freq[i]*360
        out270 = 11.302*1e-3/3e8*freq[i]*360
        

        out0 = out0-out0
        out90 = out90 - out0
        out180 = out180 - out0
        out270 = out270 - out0
        
        
        if portdeg == 0:
            array[i] = array[i] - out0
        if portdeg == 90:
            array[i] = array[i] - out90
        if portdeg == 180:
            array[i] = array[i] - out180
        if portdeg == 270:
            array[i] = array[i] - out270
        
        if array[i] > 180:
            array[i] = array[i] - 360
        elif array[i] < -180:
            array[i] = array[i] + 360
            
    return array

def returnSParameter(ampArray, freqArray, phaseArray, freqSel, phaseOffset):
    over = 0
    pi = 3.14159
    for i in range(len(freqArray)):
        if freqArray[i] < freqSel:
            over += 1
        elif (float(freqArray[i]) >= freqSel) and (over > 1):
            place = i-1
            if phaseOffset == 0:
                phase = phaseArray[place]-154.8
            else:
                phase = -phaseArray[place] + phaseOffset
            real = ampArray[place]*np.cos(phase*pi/180)
            imag = ampArray[place]*np.sin(phase*pi/180)

           
            break
            
    return ampArray[place], real, imag, phase


In [7]:
##############################Initialise#####################################################
nrow = 2
ncol = 1
xsize = 4.5
ysize = 3.5


#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))


ax[0].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
ax[0].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
ax[0].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
ax[0].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
ax[0].set_ylabel(r'S$_{11}$ (dB) - 2U', fontsize = labelfont)
ax[0].set_xlabel(r'$f$ (GHz)', fontsize = labelfont)

ax[1].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
ax[1].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
ax[1].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
ax[1].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
ax[1].set_ylabel(r'S$_{11}$ (dB) - 3U', fontsize = labelfont)
ax[1].set_xlabel(r'$f$ (GHz)', fontsize = labelfont)

#############################################################################################
#labels


#############################################################################################


freq2U, returnLoss2U, phase2U, impedanceABS2U = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_210924_114709_2U.csv")
freq3U, returnLoss3U, phase3U, impedanceABS3U = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_210924_115903_3U.csv")
freq2USim, returnLoss2USim = readSimMeasurements("S Parameter New 2U.csv")
freq3USim, returnLoss3USim = readSimMeasurements("S Parameter New 3U.csv")


p1 = ax[0].plot(np.multiply(freq2U,1e-9),returnLoss2U, linewidth = 2, color = 'dodgerblue',label='Measured - 2U Heracles')
p2 = ax[1].plot(np.multiply(freq3U,1e-9),returnLoss3U, linewidth = 2, color = 'salmon',label='Measured - 3U Heracles')


p3 = ax[0].plot(np.multiply(freq2USim,1),returnLoss2USim, "--",linewidth = 2, color = 'blue',label='Simulated - 2U Heracles')
p4 = ax[1].plot(np.multiply(freq3USim,1),returnLoss3USim, "--",linewidth = 2, color = 'red',label='Simulated - 3U Heracles')


ax[0].plot([2.428,2.428],[-40,0], "--",linewidth = 1, color = 'black')
ax[1].plot([2.428,2.428],[-40,0], "--",linewidth = 1, color = 'black')


#ax[0].set_ylim(4.45, 5.05)
#ax[1].set_ylim(2.75, 3.4)

ax[0].set_xlim(2.35, 2.55)
ax[1].set_xlim(2.35, 2.55)
ax[0].set_ylim(-40,0)
ax[1].set_ylim(-40,0)

fig.set_facecolor('w')   

#ax[0].legend([s1,s3,s5,s8,s9],["OP1","OP3","OP5","OP8","OP9"],ncol = 3)
#ax[1].legend([s2,s4,s6,s7,s10],["OP2","OP4","OP6","OP7","OP10"],ncol = 3)

matplotlib.pyplot.tight_layout()
ax[0].legend( fontsize = legfont)
ax[1].legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0, hspace=0.4)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

<IPython.core.display.Javascript object>

FileNotFoundError: [Errno 2] No such file or directory: 'VNA_210924_114709_2U.csv'

In [4]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8


#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

for i in [0,1]:
    for j in [0,1]:
        ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
        
ax[0,0].set_ylabel(r'Input Reflection Loss (dB)', fontsize = labelfont)
ax[0,1].set_ylabel(r'Output Transmission Loss (dB)', fontsize = labelfont)
ax[1,0].set_ylabel(r'Input Transmission Loss (dB)', fontsize = labelfont)
ax[1,1].set_ylabel(r'Output Reflection Loss (dB)', fontsize = labelfont)

#############################################################################################
#labels


#############################################################################################

##Port 1 - 0 degrees
freq_0_port_S11, meas_0_port_S11, phase_0_port_S11, impedanceABS_0_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_161627_0_port_S11.csv")
freq_0_port_S21, meas_0_port_S21, phase_0_port_S21, impedanceABS_0_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_161858_0_port_S21.csv")
freq_0_port_S22, meas_0_port_S22, phase_0_port_S22, impedanceABS_0_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162113_0_port_S22.csv")
freq_0_port_S12, meas_0_port_S12, phase_0_port_S12, impedanceABS_0_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162207_0_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S11,1e-6),meas_0_port_S11, linewidth = 1, color = 'dodgerblue',label='S(1,1)$_2$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_0_port_S21,1e-6),meas_0_port_S21, linewidth = 1, color = 'dodgerblue',label='S(2,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_0_port_S22,1e-6),meas_0_port_S22, linewidth = 1, color = 'dodgerblue',label='S(2,2)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_0_port_S12,1e-6),meas_0_port_S12, linewidth = 1, color = 'dodgerblue',label='S(1,2)')

##Port 2 - 90 degrees
freq_90_port_S11, meas_90_port_S11, phase_90_port_S11, impedanceABS_90_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_163407_90_port_S11.csv")
freq_90_port_S21, meas_90_port_S21, phase_90_port_S21, impedanceABS_90_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_163533_90_port_S21.csv")
freq_90_port_S22, meas_90_port_S22, phase_90_port_S22, impedanceABS_90_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162645_90_port_S22.csv")
freq_90_port_S12, meas_90_port_S12, phase_90_port_S12, impedanceABS_90_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162548_90_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_90_port_S11,1e-6),meas_90_port_S11, linewidth = 1, color = 'red',label='S(1,1)$_3$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_90_port_S21,1e-6),meas_90_port_S21, linewidth = 1, color = 'red',label='S(3,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_90_port_S22,1e-6),meas_90_port_S22, linewidth = 1, color = 'red',label='S(3,3)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S12,1e-6),meas_90_port_S12, linewidth = 1, color = 'red',label='S(1,3)')

##Port 3 - 180 degrees
freq_180_port_S11, meas_180_port_S11, phase_180_port_S11, impedanceABS_180_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164114_180_port_S11.csv")
freq_180_port_S21, meas_180_port_S21, phase_180_port_S21, impedanceABS_180_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164031_180_port_S21.csv")
freq_180_port_S22, meas_180_port_S22, phase_180_port_S22, impedanceABS_180_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164250_180_port_S22.csv")
freq_180_port_S12, meas_180_port_S12, phase_180_port_S12, impedanceABS_180_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164343_180_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_180_port_S11,1e-6),meas_180_port_S11, linewidth = 1, color = 'green',label='S(1,1)$_4$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S21,1e-6),meas_180_port_S21, linewidth = 1, color = 'green',label='S(4,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_180_port_S22,1e-6),meas_180_port_S22, linewidth = 1, color = 'green',label='S(4,4)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_180_port_S12,1e-6),meas_180_port_S12, linewidth = 1, color = 'green',label='S(1,4)')

##Port 4 - 270 degrees
freq_270_port_S11, meas_270_port_S11, phase_270_port_S11, impedanceABS_270_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164951_270_port_S11.csv")
freq_270_port_S21, meas_270_port_S21, phase_270_port_S21, impedanceABS_270_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_165030_270_port_S21.csv")
freq_270_port_S22, meas_270_port_S22, phase_270_port_S22, impedanceABS_270_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164757_270_port_S22.csv")
freq_270_port_S12, meas_270_port_S12, phase_270_port_S12, impedanceABS_270_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164722_270_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_270_port_S11,1e-6),meas_270_port_S11, linewidth = 1, color = 'goldenrod',label='S(1,1)$_5$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_270_port_S21,1e-6),meas_270_port_S21, linewidth = 1, color = 'goldenrod',label='S(5,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S22,1e-6),meas_270_port_S22, linewidth = 1, color = 'goldenrod',label='S(5,5)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_270_port_S12,1e-6),meas_270_port_S12, linewidth = 1, color = 'goldenrod',label='S(1,5)')

ax[0,0].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[1,0].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[0,1].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[1,1].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')

ax[0,0].set_ylim(-34,-16)
ax[1,0].set_ylim(-12.6, -7.5)
ax[0,1].set_ylim(-13,-7.7)
ax[1,1].set_ylim(-27,-15)

fig.set_facecolor('w')   

#ax[0].legend([s1,s3,s5,s8,s9],["OP1","OP3","OP5","OP8","OP9"],ncol = 3)
#ax[1].legend([s2,s4,s6,s7,s10],["OP2","OP4","OP6","OP7","OP10"],ncol = 3)

matplotlib.pyplot.tight_layout()
ax[0,0].legend( fontsize = legfont)
ax[1,1].legend( fontsize = legfont)
ax[0,1].legend( fontsize = legfont)
ax[1,0].legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8


#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

for i in [0,1]:
    for j in [0,1]:
        ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
        
ax[0,0].set_ylabel(r'Input Reflection Phase (dB)', fontsize = labelfont)
ax[0,1].set_ylabel(r'Output Transmission Phase (dB)', fontsize = labelfont)
ax[1,0].set_ylabel(r'Input Transmission Phase (dB)', fontsize = labelfont)
ax[1,1].set_ylabel(r'Output Reflection Phase (dB)', fontsize = labelfont)

#############################################################################################
#labels


#############################################################################################

##Port 1 - 0 degrees
freq_0_port_S11, meas_0_port_S11, phase_0_port_S11, impedanceABS_0_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_161627_0_port_S11.csv")
freq_0_port_S21, meas_0_port_S21, phase_0_port_S21, impedanceABS_0_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_161858_0_port_S21.csv")
freq_0_port_S22, meas_0_port_S22, phase_0_port_S22, impedanceABS_0_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162113_0_port_S22.csv")
freq_0_port_S12, meas_0_port_S12, phase_0_port_S12, impedanceABS_0_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162207_0_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S11,1e-6),phase_0_port_S11, linewidth = 1, color = 'dodgerblue',label='S(1,1)$_2$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_0_port_S21,1e-6),phase_0_port_S21, linewidth = 1, color = 'dodgerblue',label='S(2,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_0_port_S22,1e-6),phase_0_port_S22, linewidth = 1, color = 'dodgerblue',label='S(2,2)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_0_port_S12,1e-6),phase_0_port_S12, linewidth = 1, color = 'dodgerblue',label='S(1,2)')

##Port 2 - 90 degrees
freq_90_port_S11, meas_90_port_S11, phase_90_port_S11, impedanceABS_90_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_163407_90_port_S11.csv")
freq_90_port_S21, meas_90_port_S21, phase_90_port_S21, impedanceABS_90_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_163533_90_port_S21.csv")
freq_90_port_S22, meas_90_port_S22, phase_90_port_S22, impedanceABS_90_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162645_90_port_S22.csv")
freq_90_port_S12, meas_90_port_S12, phase_90_port_S12, impedanceABS_90_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_162548_90_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_90_port_S11,1e-6),phase_90_port_S11, linewidth = 1, color = 'red',label='S(1,1)$_3$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_90_port_S21,1e-6),phase_90_port_S21, linewidth = 1, color = 'red',label='S(3,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_90_port_S22,1e-6),phase_90_port_S22, linewidth = 1, color = 'red',label='S(3,3)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S12,1e-6),phase_90_port_S12, linewidth = 1, color = 'red',label='S(1,3)')

##Port 3 - 180 degrees
freq_180_port_S11, meas_180_port_S11, phase_180_port_S11, impedanceABS_180_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164114_180_port_S11.csv")
freq_180_port_S21, meas_180_port_S21, phase_180_port_S21, impedanceABS_180_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164031_180_port_S21.csv")
freq_180_port_S22, meas_180_port_S22, phase_180_port_S22, impedanceABS_180_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164250_180_port_S22.csv")
freq_180_port_S12, meas_180_port_S12, phase_180_port_S12, impedanceABS_180_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164343_180_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_180_port_S11,1e-6),phase_180_port_S11, linewidth = 1, color = 'green',label='S(1,1)$_4$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S21,1e-6),phase_180_port_S21, linewidth = 1, color = 'green',label='S(4,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_180_port_S22,1e-6),phase_180_port_S22, linewidth = 1, color = 'green',label='S(4,4)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_180_port_S12,1e-6),phase_180_port_S12, linewidth = 1, color = 'green',label='S(1,4)')

##Port 4 - 270 degrees
freq_270_port_S11, meas_270_port_S11, phase_270_port_S11, impedanceABS_270_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164951_270_port_S11.csv")
freq_270_port_S21, meas_270_port_S21, phase_270_port_S21, impedanceABS_270_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_165030_270_port_S21.csv")
freq_270_port_S22, meas_270_port_S22, phase_270_port_S22, impedanceABS_270_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164757_270_port_S22.csv")
freq_270_port_S12, meas_270_port_S12, phase_270_port_S12, impedanceABS_270_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20210924/","VNA_210924_164722_270_port_S12.csv")

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_270_port_S11,1e-6),phase_270_port_S11, linewidth = 1, color = 'goldenrod',label='S(1,1)$_5$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_270_port_S21,1e-6),phase_270_port_S21, linewidth = 1, color = 'goldenrod',label='S(5,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S22,1e-6),phase_270_port_S22, linewidth = 1, color = 'goldenrod',label='S(5,5)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_270_port_S12,1e-6),phase_270_port_S12, linewidth = 1, color = 'goldenrod',label='S(1,5)')

ax[0,0].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax[1,0].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax[0,1].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax[1,1].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')

ax[0,0].set_ylim(-180,180)
ax[1,0].set_ylim(-180,180)
ax[0,1].set_ylim(-180,180)
ax[1,1].set_ylim(-180,180)

fig.set_facecolor('w')   

#ax[0].legend([s1,s3,s5,s8,s9],["OP1","OP3","OP5","OP8","OP9"],ncol = 3)
#ax[1].legend([s2,s4,s6,s7,s10],["OP2","OP4","OP6","OP7","OP10"],ncol = 3)

matplotlib.pyplot.tight_layout()
ax[0,0].legend( fontsize = legfont)
ax[1,1].legend( fontsize = legfont)
ax[0,1].legend( fontsize = legfont)
ax[1,0].legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: readMiniVNAMeasurements() missing 1 required positional argument: 'portdeg'

In [6]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8


#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

for i in [0,1]:
    for j in [0,1]:
        ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
        
ax[0,0].set_ylabel(r'Input Reflection Loss (dB)', fontsize = labelfont)
ax[0,1].set_ylabel(r'Output Transmission Loss (dB)', fontsize = labelfont)
ax[1,0].set_ylabel(r'Input Transmission Loss (dB)', fontsize = labelfont)
ax[1,1].set_ylabel(r'Output Reflection Loss (dB)', fontsize = labelfont)

#############################################################################################
#labels


#############################################################################################

##Port 1 - 0 degrees

freq_0_port_S11, meas_0_port_S11, phase_0_port_S11, impedanceABS_0_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093855_S11_1.csv",0)
freq_0_port_S21, meas_0_port_S21, phase_0_port_S21, impedanceABS_0_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093802_S21.csv",0)
freq_0_port_S22, meas_0_port_S22, phase_0_port_S22, impedanceABS_0_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094031_S22.csv",0)
freq_0_port_S12, meas_0_port_S12, phase_0_port_S12, impedanceABS_0_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094105_S12.csv",0)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S11,1e-6),meas_0_port_S11, linewidth = 1, color = 'dodgerblue',label='S(1,1)$_2$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_0_port_S21,1e-6),meas_0_port_S21, linewidth = 1, color = 'dodgerblue',label='S(2,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_0_port_S22,1e-6),meas_0_port_S22, linewidth = 1, color = 'dodgerblue',label='S(2,2)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_0_port_S12,1e-6),meas_0_port_S12, linewidth = 1, color = 'dodgerblue',label='S(1,2)')

##Port 2 - 90 degrees
freq_90_port_S11, meas_90_port_S11, phase_90_port_S11, impedanceABS_90_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_092620_S11_2.csv",90)
freq_90_port_S21, meas_90_port_S21, phase_90_port_S21, impedanceABS_90_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093358_S31.csv",90)
freq_90_port_S22, meas_90_port_S22, phase_90_port_S22, impedanceABS_90_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093123_S33.csv",90)
freq_90_port_S12, meas_90_port_S12, phase_90_port_S12, impedanceABS_90_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093231_S13.csv",90)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_90_port_S11,1e-6),meas_90_port_S11, linewidth = 1, color = 'red',label='S(1,1)$_3$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_90_port_S21,1e-6),meas_90_port_S21, linewidth = 1, color = 'red',label='S(3,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_90_port_S22,1e-6),meas_90_port_S22, linewidth = 1, color = 'red',label='S(3,3)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S12,1e-6),meas_90_port_S12, linewidth = 1, color = 'red',label='S(1,3)')

##Port 3 - 180 degrees
freq_180_port_S11, meas_180_port_S11, phase_180_port_S11, impedanceABS_180_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_095000_S11_3.csv",180)
freq_180_port_S21, meas_180_port_S21, phase_180_port_S21, impedanceABS_180_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_095130_S41.csv",180)
freq_180_port_S22, meas_180_port_S22, phase_180_port_S22, impedanceABS_180_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094742_S44.csv",180)
freq_180_port_S12, meas_180_port_S12, phase_180_port_S12, impedanceABS_180_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094700_S14.csv",180)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_180_port_S11,1e-6),meas_180_port_S11, linewidth = 1, color = 'green',label='S(1,1)$_4$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S21,1e-6),meas_180_port_S21, linewidth = 1, color = 'green',label='S(4,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_180_port_S22,1e-6),meas_180_port_S22, linewidth = 1, color = 'green',label='S(4,4)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_180_port_S12,1e-6),meas_180_port_S12, linewidth = 1, color = 'green',label='S(1,4)')

##Port 4 - 270 degrees
freq_270_port_S11, meas_270_port_S11, phase_270_port_S11, impedanceABS_270_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100803_S11_4.csv",270)
freq_270_port_S21, meas_270_port_S21, phase_270_port_S21, impedanceABS_270_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100654_S51.csv",270)
freq_270_port_S22, meas_270_port_S22, phase_270_port_S22, impedanceABS_270_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100914_S55.csv",270)
freq_270_port_S12, meas_270_port_S12, phase_270_port_S12, impedanceABS_270_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_101008_S15.csv",270)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_270_port_S11,1e-6),meas_270_port_S11, linewidth = 1, color = 'goldenrod',label='S(1,1)$_5$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_270_port_S21,1e-6),meas_270_port_S21, linewidth = 1, color = 'goldenrod',label='S(5,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S22,1e-6),meas_270_port_S22, linewidth = 1, color = 'goldenrod',label='S(5,5)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_270_port_S12,1e-6),meas_270_port_S12, linewidth = 1, color = 'goldenrod',label='S(1,5)')

ax[0,0].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[1,0].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[0,1].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[1,1].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')

ax[0,0].set_ylim(-30,-19)
ax[1,0].set_ylim(-10, -5)
ax[0,1].set_ylim(-10, -5)
ax[1,1].set_ylim(-32,-15)

fig.set_facecolor('w')   

#ax[0].legend([s1,s3,s5,s8,s9],["OP1","OP3","OP5","OP8","OP9"],ncol = 3)
#ax[1].legend([s2,s4,s6,s7,s10],["OP2","OP4","OP6","OP7","OP10"],ncol = 3)

matplotlib.pyplot.tight_layout()
ax[0,0].legend( fontsize = legfont)
ax[1,1].legend( fontsize = legfont)
ax[0,1].legend( fontsize = legfont)
ax[1,0].legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: too many values to unpack (expected 4)

In [15]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8


#############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

for i in [0,1]:
    for j in [0,1]:
        ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
        
ax[0,0].set_ylabel(r'Port 2 Isolation (dB)', fontsize = labelfont)
ax[0,1].set_ylabel(r'Port 3 Isolation (dB)', fontsize = labelfont)
ax[1,0].set_ylabel(r'Port 4 Isolation (dB)', fontsize = labelfont)
ax[1,1].set_ylabel(r'Port 5 Isolation (dB)', fontsize = labelfont)

#############################################################################################
#labels


#############################################################################################

##Port 1 - 0 degrees

freq_0_port_S11, meas_0_port_S11, phase_0_port_S11, impedanceABS_0_port_S11, impedanceImag1 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105123_S32.csv",0)
freq_0_port_S21, meas_0_port_S21, phase_0_port_S21, impedanceABS_0_port_S21, impedanceImag2 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105013_S42.csv",0)
freq_0_port_S22, meas_0_port_S22, phase_0_port_S22, impedanceABS_0_port_S22, impedanceImag3 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105944_S52.csv",0)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S11,1e-6),meas_0_port_S11, linewidth = 1, color = 'dodgerblue',label='S(2,3)')
ITP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S21,1e-6),meas_0_port_S21, linewidth = 1, color = 'red',label='S(2,4)')
ORP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S22,1e-6),meas_0_port_S22, linewidth = 1, color = 'green',label='S(2,5)')

##Port 2 - 90 degrees
freq_90_port_S11, meas_90_port_S11, phase_90_port_S11, impedanceABS_90_port_S11, impedanceImag4 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105123_S32.csv",0)
freq_90_port_S21, meas_90_port_S21, phase_90_port_S21, impedanceABS_90_port_S21, impedanceImag5 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105458_S34.csv",0)
freq_90_port_S22, meas_90_port_S22, phase_90_port_S22, impedanceABS_90_port_S22, impedanceImag6 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105623_S35.csv",0)

IRP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S11,1e-6),meas_90_port_S11, linewidth = 1, color = 'dodgerblue',label='S(3,2)')
ITP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S21,1e-6),meas_90_port_S21, linewidth = 1, color = 'red',label='S(3,4)')
ORP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S22,1e-6),meas_90_port_S22, linewidth = 1, color = 'green',label='S(3,5)')

##Port 3 - 180 degrees
freq_180_port_S11, meas_180_port_S11, phase_180_port_S11, impedanceABS_180_port_S11, impedanceImag7 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105013_S42.csv",0)
freq_180_port_S21, meas_180_port_S21, phase_180_port_S21, impedanceABS_180_port_S21, impedanceImag8 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_104815_S43.csv",0)
freq_180_port_S22, meas_180_port_S22, phase_180_port_S22, impedanceABS_180_port_S22, impedanceImag9 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_103000_S45.csv",0)

IRP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S11,1e-6),meas_180_port_S11, linewidth = 1, color = 'dodgerblue',label='S(4,2)')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S21,1e-6),meas_180_port_S21, linewidth = 1, color = 'red',label='S(4,3)')
ORP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S22,1e-6),meas_180_port_S22, linewidth = 1, color = 'green',label='S(4,5)')

##Port 4 - 270 degrees
freq_270_port_S11, meas_270_port_S11, phase_270_port_S11, impedanceABS_270_port_S11, impedanceImag10 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105944_S52.csv",0)
freq_270_port_S21, meas_270_port_S21, phase_270_port_S21, impedanceABS_270_port_S21, impedanceImag11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105802_S53.csv",0)
freq_270_port_S22, meas_270_port_S22, phase_270_port_S22, impedanceABS_270_port_S22, impedanceImag12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_102733_S54.csv",0)

IRP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S11,1e-6),meas_270_port_S11, linewidth = 1, color = 'dodgerblue',label='S(5,2)')
ITP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S21,1e-6),meas_270_port_S21, linewidth = 1, color = 'red',label='S(5,3)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S22,1e-6),meas_270_port_S22, linewidth = 1, color = 'green',label='S(5,4)')

ax[0,0].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[1,0].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[0,1].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')
ax[1,1].plot([437.825,437.825],[-50,0], "--",linewidth = 1, color = 'black')

upper = -10
lower = -20
ax[0,0].set_ylim(lower,upper)
ax[1,0].set_ylim(lower, upper)
ax[0,1].set_ylim(lower, upper)
ax[1,1].set_ylim(lower,upper)

fig.set_facecolor('w')   

#ax[0].legend([s1,s3,s5,s8,s9],["OP1","OP3","OP5","OP8","OP9"],ncol = 3)
#ax[1].legend([s2,s4,s6,s7,s10],["OP2","OP4","OP6","OP7","OP10"],ncol = 3)

matplotlib.pyplot.tight_layout()
ax[0,0].legend( fontsize = legfont)
ax[1,1].legend( fontsize = legfont)
ax[0,1].legend( fontsize = legfont)
ax[1,0].legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
##############################Initialise#####################################################
nrow = 2
ncol = 2
xsize = 8
ysize = 8


############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))

for i in [0,1]:
    for j in [0,1]:
        ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i,j].set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
        
ax[0,0].set_ylabel(r'Input Reflection Phase ($^\circ$)', fontsize = labelfont)
ax[0,1].set_ylabel(r'Output Transmission Phase ($^\circ$)', fontsize = labelfont)
ax[1,0].set_ylabel(r'Input Transmission Phase ($^\circ$)', fontsize = labelfont)
ax[1,1].set_ylabel(r'Output Reflection Phase ($^\circ$)', fontsize = labelfont)

#############################################################################################
#labels


#############################################################################################

##Port 1 - 0 degrees
freq_0_port_S11, meas_0_port_S11, phase_0_port_S11, impedanceABS_0_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093855_S11_1.csv",0)
freq_0_port_S21, meas_0_port_S21, phase_0_port_S21, impedanceABS_0_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093802_S21.csv",0)
freq_0_port_S22, meas_0_port_S22, phase_0_port_S22, impedanceABS_0_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094031_S22.csv",0)
freq_0_port_S12, meas_0_port_S12, phase_0_port_S12, impedanceABS_0_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094105_S12.csv",0)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_0_port_S11,1e-6),phase_0_port_S11, linewidth = 1, color = 'dodgerblue',label='S(1,1)$_2$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_0_port_S21,1e-6),phase_0_port_S21, linewidth = 1, color = 'dodgerblue',label='S(2,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_0_port_S22,1e-6),phase_0_port_S22, linewidth = 1, color = 'dodgerblue',label='S(2,2)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_0_port_S12,1e-6),phase_0_port_S12, linewidth = 1, color = 'dodgerblue',label='S(1,2)')

##Port 2 - 90 degrees
freq_90_port_S11, meas_90_port_S11, phase_90_port_S11, impedanceABS_90_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_092620_S11_2.csv",90)
freq_90_port_S21, meas_90_port_S21, phase_90_port_S21, impedanceABS_90_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093358_S31.csv",90)
freq_90_port_S22, meas_90_port_S22, phase_90_port_S22, impedanceABS_90_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093123_S33.csv",90)
freq_90_port_S12, meas_90_port_S12, phase_90_port_S12, impedanceABS_90_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093231_S13.csv",90)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_90_port_S11,1e-6),phase_90_port_S11, linewidth = 1, color = 'red',label='S(1,1)$_3$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_90_port_S21,1e-6),phase_90_port_S21, linewidth = 1, color = 'red',label='S(3,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_90_port_S22,1e-6),phase_90_port_S22, linewidth = 1, color = 'red',label='S(3,3)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_90_port_S12,1e-6),phase_90_port_S12, linewidth = 1, color = 'red',label='S(1,3)')

##Port 3 - 180 degrees
freq_180_port_S11, meas_180_port_S11, phase_180_port_S11, impedanceABS_180_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_095000_S11_3.csv",180)
freq_180_port_S21, meas_180_port_S21, phase_180_port_S21, impedanceABS_180_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_095130_S41.csv",180)
freq_180_port_S22, meas_180_port_S22, phase_180_port_S22, impedanceABS_180_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094742_S44.csv",180)
freq_180_port_S12, meas_180_port_S12, phase_180_port_S12, impedanceABS_180_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094700_S14.csv",180)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_180_port_S11,1e-6),phase_180_port_S11, linewidth = 1, color = 'green',label='S(1,1)$_4$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_180_port_S21,1e-6),phase_180_port_S21, linewidth = 1, color = 'green',label='S(4,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_180_port_S22,1e-6),phase_180_port_S22, linewidth = 1, color = 'green',label='S(4,4)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_180_port_S12,1e-6),phase_180_port_S12, linewidth = 1, color = 'green',label='S(1,4)')

##Port 4 - 270 degrees
freq_270_port_S11, meas_270_port_S11, phase_270_port_S11, impedanceABS_270_port_S11 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100803_S11_4.csv",270)
freq_270_port_S21, meas_270_port_S21, phase_270_port_S21, impedanceABS_270_port_S21 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100654_S51.csv",270)
freq_270_port_S22, meas_270_port_S22, phase_270_port_S22, impedanceABS_270_port_S22 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100914_S55.csv",270)
freq_270_port_S12, meas_270_port_S12, phase_270_port_S12, impedanceABS_270_port_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_101008_S15.csv",270)

IRP_PORT1 = ax[0,0].plot(np.multiply(freq_270_port_S11,1e-6),phase_270_port_S11, linewidth = 1, color = 'goldenrod',label='S(1,1)$_5$')
ITP_PORT1 = ax[1,0].plot(np.multiply(freq_270_port_S21,1e-6),phase_270_port_S21, linewidth = 1, color = 'goldenrod',label='S(5,1)')
ORP_PORT1 = ax[1,1].plot(np.multiply(freq_270_port_S22,1e-6),phase_270_port_S22, linewidth = 1, color = 'goldenrod',label='S(5,5)$_1$')
OTP_PORT1 = ax[0,1].plot(np.multiply(freq_270_port_S12,1e-6),phase_270_port_S12, linewidth = 1, color = 'goldenrod',label='S(1,5)')

ax[0,0].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax[1,0].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax[0,1].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax[1,1].plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')

ax[0,0].set_ylim(-180,180)
ax[1,0].set_ylim(-180,180)
ax[0,1].set_ylim(-180,180)
ax[1,1].set_ylim(-180,180)

center = 625
print(freq_270_port_S11[center])
print("S12 - PHASE: /n")

print(-phase_0_port_S12[center]+phase_0_port_S12[center])
print(-phase_90_port_S12[center]+phase_0_port_S12[center])
print(-phase_180_port_S12[center]+phase_0_port_S12[center])
print(-phase_270_port_S12[center]+phase_0_port_S12[center])

fig.set_facecolor('w')   

#ax[0].legend([s1,s3,s5,s8,s9],["OP1","OP3","OP5","OP8","OP9"],ncol = 3)
#ax[1].legend([s2,s4,s6,s7,s10],["OP2","OP4","OP6","OP7","OP10"],ncol = 3)

matplotlib.pyplot.tight_layout()
ax[0,0].legend( fontsize = legfont)
ax[1,1].legend( fontsize = legfont)
ax[0,1].legend( fontsize = legfont)
ax[1,0].legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

437765625.0
S12 - PHASE: /n
0.0
88.23509423125
176.92684430625
269.4471525125


In [162]:
freq_S12, meas_S12, phase_S12, impedanceABS_S12 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094105_S12.csv",0)
freq_S13, meas_S13, phase_S13, impedanceABS_S13 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093231_S13.csv",90)
freq_S14, meas_S14, phase_S14, impedanceABS_S14 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094700_S14.csv",180)
freq_S15, meas_S15, phase_S15, impedanceABS_S15 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_101008_S15.csv",270)
phaseArray0, amp, real, imag, phase = returnSParameter(meas_S12, freq_S12, phase_S12, float(437.875e6), 0)
print(phaseArray)
print(amp)
print(real)
print(imag)
print(phase)
phaseArray, amp, real, imag, phase = returnSParameter(meas_S13, freq_S13, phase_S13, float(437.875e6), phaseArray0)
print(phaseArray)
print(amp)
print(real)
print(imag)
print(phase)
phaseArray, amp, real, imag, phase = returnSParameter(meas_S14, freq_S14, phase_S14, float(437.875e6), phaseArray0)
print(phaseArray)
print(amp)
print(real)
print(imag)
print(phase)
phaseArray, amp, real, imag, phase = returnSParameter(meas_S15, freq_S15, phase_S15, float(437.875e6), phaseArray0)
print(phaseArray)
print(amp)
print(real)
print(imag)
print(phase)

-114.66715251250001
-8.02
-0.359330218303947
-0.16924219681737968
-154.78
66.54490576875
-8.46
0.011629165234856187
0.3773930601854772
88.23509423125
-22.14684430625
-8.28
-0.38492394649466144
0.02066687309865062
176.92684430625
-114.66715251250001
-8.43
-0.0036572505477184647
-0.3788608819260664
269.4471525125


In [35]:
##############################Initialise#####################################################
nrow = 1
ncol = 1
xsize = 7
ysize = 4


############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))
if (nrow == 1) and (ncol == 1):
    ax.tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    ax.tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    ax.tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    ax.tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
elif ((nrow != 1) and (ncol == 1)) or ((nrow == 1) and (ncol != 1)):
    for i in [0,1]:
        ax[i].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
else:
    for i in [0,1]:
        for j in [0,1]:
            ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
            ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
            ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
            ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    
    

ax.set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
ax.set_ylabel(r'Return Loss (dB)', fontsize = labelfont)\

#############################################################################################
#labels


#############################################################################################

freq_A1, meas_A1, phase_A1, impedanceReal_A1, impedanceImag_A1 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_1624515_0deg_antenna_meas_3.csv",0)
freq_A2, meas_A2, phase_A2, impedanceReal_A2, impedanceImag_A2 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_163006_90deg_antenna_meas_3.csv",0)
freq_A3, meas_A3, phase_A3, impedanceReal_A3, impedanceImag_A3 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_163123_180deg_antenna_meas_3.csv",0)
freq_A4, meas_A4, phase_A4, impedanceReal_A4, impedanceImag_A4 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_162705_270deg_antenna_meas_3.csv",0)

A1 = ax.plot(np.multiply(freq_A1,1e-6),meas_A1, linewidth = 1, color = 'dodgerblue',label='0$^\circ$ Antenna')
A2 = ax.plot(np.multiply(freq_A2,1e-6),meas_A2, linewidth = 1, color = 'red',label='90$^\circ$ Antenna')
A3 = ax.plot(np.multiply(freq_A3,1e-6),meas_A3, linewidth = 1, color = 'green',label='180$^\circ$ Antenna')
A4 = ax.plot(np.multiply(freq_A4,1e-6),meas_A4, linewidth = 1, color = 'goldenrod',label='270$^\circ$ Antenna')



ax.plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax.plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax.plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')
ax.plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')

ax.set_ylim(-45,-10)

fig.set_facecolor('w')   

matplotlib.pyplot.tight_layout()
ax.legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
print(impedanceReal_A1[625])
print(impedanceImag_A1[625])
print("\n")
print(impedanceReal_A2[625])
print(impedanceImag_A2[625])
print("\n")
print(impedanceReal_A3[625])
print(impedanceImag_A3[625])
print("\n")
print(impedanceReal_A4[625])
print(impedanceImag_A4[625])
print("\n")

print(meas_A1[625])
print("\n")
print(meas_A2[625])
print("\n")
print(meas_A3[625])
print("\n")
print(meas_A4[625])
print("\n")

53.5
-3.6


51.8
-0.4


51.3
-2.2


53.8
-2.3


-26.3


-35.06


-31.81


-27.41




In [21]:
##############################Initialise#####################################################
nrow = 1
ncol = 1
xsize = 7
ysize = 4


############################################################################################
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

#set global plotting parameters
linew = 5.0 #line width
#msize = 14 #marker size
axew = 1.0 #axes width

#set the distance to offset the numbers from the ticks
numpad = 10

#set global tick parameters
majw = axew #major tick width
majl = 2.0 #major tick length
minw = axew #minor tick width
minl = 2.0 #minor tick length

#set global font sizes
axefont = 12 #axis label font size
numsize = 12 #axis number font size
legfont = 12 #legend font size
labelfont=16

fig, ax = plt.subplots(nrows=nrow, ncols=ncol, figsize=(xsize,ysize))
if (nrow == 1) and (ncol == 1):
    ax.tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    ax.tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
    ax.tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    ax.tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
elif ((nrow != 1) and (ncol == 1)) or ((nrow == 1) and (ncol != 1)):
    for i in [0,1]:
        ax[i].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
        ax[i].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
        ax[i].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
else:
    for i in [0,1]:
        for j in [0,1]:
            ax[i,j].tick_params(axis='x',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
            ax[i,j].tick_params(axis='x',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,top='off')
            ax[i,j].tick_params(axis='y',which='major',width=majw,length=majl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
            ax[i,j].tick_params(axis='y',which='minor',width=minw,length=minl,labelsize=numsize,zorder=1,direction='in',pad=numpad,right='off')
    
    

ax.set_xlabel(r'$f$ (MHz)', fontsize = labelfont)
ax.set_ylabel(r'Return Loss (dB)', fontsize = labelfont)\

#############################################################################################
#labels


#############################################################################################

freq_A1, meas_A1, phase_A1, impedanceABS_A1 = readMiniVNAMeasurements("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211015_093029_finalassembly_RL.csv",0)

A1 = ax.plot(np.multiply(freq_A1,1e-6),meas_A1, linewidth = 1, color = 'dodgerblue',label='0$^\circ$ Antenna')



ax.plot([437.825,437.825],[-180,180], "--",linewidth = 1, color = 'black')

ax.set_ylim(-45,-10)

fig.set_facecolor('w')   

matplotlib.pyplot.tight_layout()
ax.legend( fontsize = legfont)
matplotlib.pyplot.tight_layout()


fig.subplots_adjust(wspace=0.3, hspace=0.2)
#plt.savefig("SBAND_Measurement_Comparison.png", dpi=400)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
freq_S11, meas_S11, phase_S11, impedanceRel_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093855_S11_1.csv",0)
freq_S12, meas_S12, phase_S12, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094105_S12.csv",0)
freq_S11, meas_S13, phase_S13, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093231_S13.csv",90)
freq_S11, meas_S14, phase_S14, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094700_S14.csv",180)
freq_S11, meas_S15, phase_S15, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_101008_S15.csv",270)
freq_S11, meas_S21, phase_S21, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093802_S21.csv",0)
freq_S11, meas_S22, phase_S22, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094031_S22.csv",0)
freq_S11, meas_S23, phase_S23, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105123_S32.csv",0)
freq_S11, meas_S24, phase_S24, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105013_S42.csv",0)
freq_S11, meas_S25, phase_S25, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105944_S52.csv",0)
freq_S11, meas_S31, phase_S31, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093358_S31.csv",90)
freq_S11, meas_S32, phase_S32, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105123_S32.csv",0)
freq_S11, meas_S33, phase_S33, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_093123_S33.csv",0)
freq_S11, meas_S34, phase_S34, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105458_S34.csv",0)
freq_S11, meas_S35, phase_S35, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105623_S35.csv",0)
freq_S11, meas_S41, phase_S41, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_095130_S41.csv",180)
freq_S11, meas_S42, phase_S42, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105013_S42.csv",0)
freq_S11, meas_S43, phase_S43, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_104815_S43.csv",0)
freq_S11, meas_S44, phase_S44, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_094742_S44.csv",0)
freq_S11, meas_S45, phase_S45, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_103000_S45.csv",0)
freq_S11, meas_S51, phase_S51, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100654_S51.csv",270)
freq_S11, meas_S52, phase_S52, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105944_S52.csv",0)
freq_S11, meas_S53, phase_S53, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_105802_S53.csv",0)
freq_S11, meas_S54, phase_S54, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_102733_S54.csv",0)
freq_S11, meas_S55, phase_S55, impedanceABS_A1, impedanceReal_A1 = readMiniVNAMeasurementsSParam("C:/Users/nicho/Desktop/AlbertaSat_Non_SVN/UHF/Arke-20211014/","VNA_211014_100914_S55.csv",0)

print("|S|, real, imag, phaseDeg")
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S11, freq_S11, phase_S11, 437.825e6, 0))
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S12, freq_S11, phase_S12, 437.825e6, 0))
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S13, freq_S11, phase_S13, 437.825e6, 0))
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S14, freq_S11, phase_S14, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S15, freq_S11, phase_S15, 437.825e6, 0)[:])
print("/n")
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S21, freq_S11, phase_S21, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S22, freq_S11, phase_S22, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S23, freq_S11, phase_S23, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S24, freq_S11, phase_S24, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S25, freq_S11, phase_S25, 437.825e6, 0)[:])
print("/n")
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S31, freq_S11, phase_S31, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S32, freq_S11, phase_S32, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S33, freq_S11, phase_S33, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S34, freq_S11, phase_S34, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S35, freq_S11, phase_S35, 437.825e6, 0)[:])
print("/n")
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S41, freq_S11, phase_S41, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S42, freq_S11, phase_S42, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S43, freq_S11, phase_S43, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S44, freq_S11, phase_S44, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S45, freq_S11, phase_S45, 437.825e6, 0)[:])
print("/n")
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S51, freq_S11, phase_S51, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S52, freq_S11, phase_S52, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S53, freq_S11, phase_S53, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S54, freq_S11, phase_S54, 437.825e6, 0)[:])
print("%.3f,%.3f, %.3f, %.1f"%returnSParameter(meas_S55, freq_S11, phase_S55, 437.825e6, 0)[:])

|S|, real, imag, phaseDeg
0.098,-0.034, 0.092, -249.5
0.397,0.397, -0.000, -0.0
0.378,0.011, -0.377, -88.3
0.385,-0.385, -0.021, -176.9
0.379,-0.004, 0.379, -269.5
/n
0.402,0.402, -0.010, -1.4
0.066,0.007, 0.066, -275.8
0.139,-0.118, 0.072, -211.5
0.217,-0.047, -0.212, -102.4
0.175,-0.169, 0.042, -193.9
/n
0.363,-0.000, -0.363, -90.1
0.139,-0.118, 0.072, -211.5
0.147,-0.100, 0.108, -227.4
0.174,-0.173, 0.021, -187.0
0.157,0.016, 0.156, -275.7
/n
0.408,-0.407, -0.036, -174.9
0.217,-0.047, -0.212, -102.4
0.185,-0.185, 0.009, -182.8
0.077,0.000, 0.077, -270.2
0.157,-0.151, 0.043, -195.8
/n
0.363,-0.002, 0.363, -269.7
0.175,-0.169, 0.042, -193.9
0.156,0.012, 0.155, -274.5
0.147,-0.139, 0.047, -198.8
0.147,-0.098, 0.110, -228.1
